In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# original video
video = cv2.VideoCapture('videos/road_video.mp4')

# will get all frames of a video
while True:
  # see if video was read and get video frame by frame
  read, frame = video.read()

  # show each frame of video
  if not read:
    break;

  cv2.imshow('video', frame)

  # cancel out of video so it doesn't go infinity
  if cv2.waitKey(20) & 0xFF == ord('q'):
    break

video.release()
cv2.destroyAllWindows()


In [3]:
# function definition that creates triangle roi of current road
def region_of_interest(road, vertices):

  # creates black screen with same shape as input road image
  # no need to specify dimensions of road image like np.zeros
  mask = np.zeros_like(road)

  # get number of color channels in image
  # (255,) * channels creates a white color matching the image format
  # ie. (255,255,255) for RGB and (255,) for grayscale 
  if len(road.shape) == 3:
    match_mask_color = (255,) * road.shape[2]

  else:
    match_mask_color = 255
    
  # fill roi with white using to defined vertices
  cv2.fillPoly(mask, vertices, match_mask_color)

  # fills in the roi with the actual image of the road
  # only pixels inside the roi will be shown, everything else is black
  masked_image = cv2.bitwise_and(road, mask)
  return masked_image

height, width, channels = frame.shape

roi_vertices = [
  (0, height),
  (width / 2, height / 2),
  (width, height),
]

In [5]:
# Grayscale video
# Gaussian blurred video
# Canny Edge video
# Erosion Video --> easier to see lines of roads when thicker than canny edge

# store video file as object
video = cv2.VideoCapture('videos/road_video.mp4')

# will always return true
while True:
  # gets both the frame and whether the frame is able to be read (bool)
  read, frame = video.read() 

  # check if video can read each frame
  if not read:
    break;

  # convert to grayscale
  frame_grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

  #convert to gaussian blue
  frame_gaussian = cv2.GaussianBlur(frame_grey, (11,11), 0)

  # convert to canny edge
  lower = 10
  upper = 90
  frame_canny = cv2.Canny(frame_gaussian, lower, upper)

  # convert to dilation / erosion
  kernel = np.ones((3,3), np.uint8)
  frame_dilation = cv2.dilate(frame_canny, kernel, 1)
  frame_erosion = cv2.erode(frame_dilation, kernel, 1)

  # only able to stack videos with same color channels and dimensions
  # all put in same 1 color channel (grayscale-based)
  row1 = np.hstack((frame_grey, frame_gaussian))
  row2 = np.hstack((frame_canny, frame_dilation))
  stacked = np.vstack((row1, row2))
  cv2.imshow('Combined videos', stacked)

  # exiting video
  if cv2.waitKey(20) & 0xFF == ord('q'):
    break

# ensures video is no longer available for use
video.release()
cv2.destroyAllWindows()


In [6]:
# read video as object
video = cv2.VideoCapture('videos/road_video.mp4')

# only want the frame and whether it can be read (bool)
# purpose is to select roi for ALL frames later on
read, frame = video.read()

height, width, channels = frame.shape
roi_vertices = [
  (0, height),
  (width / 2, height / 2),
  (width, height),
]

# will always be true
while True:
  read, frame = video.read()
  # frame is able to be read successfully
  # basically this allows us to read all the frames, unless there is a false bool
  if not read:  
    break;  

  # roi with grayscale
  frame_grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

  # roi with gaussian blur
  frame_gaussian = cv2.GaussianBlur(frame_grey, (11,11), 0)

  # roi with canny edge
  lower = 10
  upper = 90
  frame_canny = cv2.Canny(frame_gaussian, lower, upper)

  # roi with dilation / erosion
  # THIS is most important step for getting best line detection when performing hough line transforms 
  kernel = np.ones((3,3), np.uint8)
  frame_dilation = cv2.dilate(frame_canny, kernel, 1)
  frame_erosion = cv2.erode(frame_dilation, kernel, 1)

  # after detecting all possible edges, get the edges only within ROI
  video_roi = region_of_interest(frame_dilation, np.array([roi_vertices], np.int32))

  # get copy of original frame, so we can append detected lines on each frame
  # we don't want to display the same frames with lines later when video plays
  og_with_lines = np.copy(frame)

  # get subset of lines possible through probablistic hough transform
  lines = cv2.HoughLinesP(video_roi, 1, np.pi/180, 20, minLineLength=5)

  # get dimensions from all lines detected
  for line in lines:
    x1, y1, x2, y2 = line[0]

    # draws the detected lines in red onto original image
    cv2.line(og_with_lines, (x1, y1), (x2, y2), (0,0,255), 2)

  # display video where original frame and frame with hough lines show together
  stacked = np.vstack((frame, og_with_lines))
  cv2.imshow('Combined videos', stacked)

  # able to quit video
  if cv2.waitKey(20) & 0xFF == ord('q'):
    break

# get out of video and window
video.release()
cv2.destroyAllWindows()

In [9]:
# original video
# capture how many frames in video
# and how many frames per second
video = cv2.VideoCapture('videos/road_video.mp4')
frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
fps = video.get(cv2.CAP_PROP_FPS)
print(frame_count)
print(fps)



1392.0
30.0
